In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import theano
import exoplanet as xo

from radvel._kepler import kepler_array as radvel_kepler
from batman._rsky import _getf as batman_kepler

%run notebook_setup

In [ ]:
def esolve_markley(M, ecc):
    M = M % (2*pi)
    if ecc == 0:
        return M
    if M > pi:
        M = 2*pi - M
        flip = True
    else:
        flip = False
    alpha = (3*pi + 1.6*(pi-abs(M))/(1+ecc) )/(pi - 6/pi)
    d = 3*(1 - ecc) + alpha*ecc
    r = 3*alpha*d * (d-1+ecc)*M + M**3
    q = 2*alpha*d*(1-ecc) - M**2
    w = (abs(r) + sqrt(q**3 + r**2))**(2/3)
    E = (2*r*w/(w**2 + w*q + q**2) + M) / d
    f_0 = E - ecc*sin(E) - M
    f_1 = 1 - ecc*cos(E)
    f_2 = ecc*sin(E)
    f_3 = 1-f_1
    d_3 = -f_0/(f_1 - 0.5*f_0*f_2/f_1)
    d_4 = -f_0/(f_1 + 0.5*d_3*f_2 + (d_3**2)*f_3/6)
    E = E -f_0/(f_1 + 0.5*d_4*f_2 + d_4**2*f_3/6 - d_4**3*f_2/24)
    if flip:
        E =  2*pi - E
    return E

In [ ]:
np.random.seed(12345)
E = np.random.uniform(-np.pi, np.pi, 5000)

evals = np.linspace(0.01, 0.99, 10)

times_rv_mu = np.empty_like(evals)
times_rv_sd = np.empty_like(evals)
err_rv = np.empty((len(evals), 4))
times_ml_mu = np.empty_like(evals)
times_ml_sd = np.empty_like(evals)
err_ml = np.empty((len(evals), 4))
times_xo_mu = np.empty_like(evals)
times_xo_sd = np.empty_like(evals)
err_xo = np.empty((len(evals), 4))
times_bm_mu = np.empty_like(evals)
times_bm_sd = np.empty_like(evals)
err_bm = np.empty((len(evals), 4))

for ie, e in enumerate(evals):
    M = E - e * np.sin(E)
    
    print("batman...")
    E0 = 2.*np.arctan(np.sqrt((1. - e)/(1. + e)))
    M0 = E0 - e*np.sin(E0);
    t = 0.5*(M - M0)/np.pi
    f_bm = batman_kepler(t, 0.0, 1.0, 1.0, 90.0, e, 0.0, 1, 1)
    E_bm = 2*np.arctan(np.sqrt((1-e)/(1+e))*np.tan(0.5*f_bm))
    assert np.all(np.isfinite(E_bm))
    results = %timeit -o batman_kepler(t, 0.0, 1.0, 1.0, 90.0, e, 0.0, 1, 1)
    times_bm_mu[ie] = np.mean(np.log10(results.all_runs)) - np.log10(results.loops)
    times_bm_sd[ie] = np.std(np.log10(results.all_runs))
    err = np.abs(E_bm - E)
    err_bm[ie] = np.percentile(err, [16, 50, 84, 90])
    
    print("radvel...")    
    E_rv = radvel_kepler(M, e)
    assert np.all(np.isfinite(E_rv))
    results = %timeit -o radvel_kepler(M, e)
    times_rv_mu[ie] = np.mean(np.log10(results.all_runs)) - np.log10(results.loops)
    times_rv_sd[ie] = np.std(np.log10(results.all_runs))
    err = np.abs(E_rv - E)
    err_rv[ie] = np.percentile(err, [16, 50, 84, 90])
    
    print("markley...")    
    E_ml = esolve_markley(M, e)
    assert np.all(np.isfinite(E_ml))
    results = %timeit -o esolve_markley(M, e)
    times_ml_mu[ie] = np.mean(np.log10(results.all_runs)) - np.log10(results.loops)
    times_ml_sd[ie] = np.std(np.log10(results.all_runs))
    err = np.abs(E_ml - E)
    err_ml[ie] = np.percentile(err, [16, 50, 84, 90])
    
    print("exoplanet...")
    op = xo.theano_ops.kepler.KeplerOp()
    func = theano.function([], op(M, e + np.zeros_like(M))[0])
    E_xo = func()
    assert np.all(np.isfinite(E_xo))
    results = %timeit -o func()
    times_xo_mu[ie] = np.mean(np.log10(results.all_runs)) - np.log10(results.loops)
    times_xo_sd[ie] = np.std(np.log10(results.all_runs))
    err = np.abs(E_xo - E)
    err_xo[ie] = np.percentile(err, [16, 50, 84, 90])

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(7, 7), sharex=True)

ax = axes[0]
ax.plot(evals, times_xo_mu - np.log10(len(E)), "o-", label="exoplanet")
ax.plot(evals, times_rv_mu - np.log10(len(E)), "o-", label="radvel")
ax.plot(evals, times_bm_mu - np.log10(len(E)), "o-", label="batman")
ax.set_ylabel(r"$\log_{10}$(runtime / sec)")
ax.yaxis.set_label_coords(-0.1, 0.5)
ax.legend(fontsize=12, ncol=3)

ax = axes[1]
plt.plot(evals, np.log10(err_xo[:, 3]), "o-", label="exoplanet")
plt.plot(evals, np.log10(err_rv[:, 3]), "o-", label="radvel")
plt.plot(evals, np.log10(err_bm[:, 3]), "o-", label="batman")
ax.set_xlim(0, 1)
ax.set_ylabel("90th percentile of\n $\log_{10}|E_\mathrm{calc} - E_\mathrm{true}|$")
ax.yaxis.set_label_coords(-0.1, 0.5)
ax.set_xlabel("eccentricity")

fig.subplots_adjust(hspace=0.05)
fig.savefig("kepler_solver.pdf", bbox_inches="tight");